# SalonEverywhere Technical Test

This is my technical test as part of my application to SalonEverywhere as a Data Science/ML Intern. 

The task can be found here: https://docs.google.com/document/d/16ECQjX29XOa3oN6qUyADK-QmSd4gYM_uaoLdYQs8w34/edit

The data can be found here: https://drive.google.com/drive/folders/1gef1LuISenQ25YsggnmcWA9nw8UUkPRp?usp=sharing

The data was too large to upload to github, however is installed locally.

Let's start by importing the libraries we'll need.

In [1]:
import numpy as np
import pandas as pd
import re
import geograpy #Extracting locations

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

from gensim.parsing.preprocessing import remove_stopwords
import gensim
stopwords = gensim.parsing.preprocessing.STOPWORDS

import canada_provinces as cp
import us_states as us

import ast

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Quinn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


I first want to see what kind of data I'm working with, so I'll continue by importing the csv files and taking a look at the contents.

In [2]:
user_profile = pd.read_csv('user_details_v1.csv')

In [3]:
#Received runtime error if both csvs are read in the same file (takes too long), so it is split
user_post = pd.read_csv('user_posts_v1.csv')

C:\Users\Quinn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,7,10,11,14,16,17,20,22,23,29,30,31,32,34,35,36,37,38,40,49,50,61,88,90,91,92,103,104,105,107,108,109,110,111,112,116,119,120,123,124,128,132,133,135,136,138,139,140,141,142,143,145,150,154,155,157,158,160,161,162,163,164,165,167,171,172,173,174,175,176,178,179,181,182,183,184,185,186,188,189,191,192,193,195,196,197,198,199,201,205,206,208,209,210,211,212,223,224,227,228,229,230,231,232,233,238,241,245,247,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Let's take a look at user profile first, and see where we can extract some data that we need.

In [4]:
user_profile.columns.values

array(['input.session_id', 'input.user_id', 'user.pk', 'user.username',
       'user.full_name', 'user.is_private', 'user.profile_pic_url',
       'user.profile_pic_id', 'user.is_verified',
       'user.has_anonymous_profile_picture', 'user.media_count',
       'user.geo_media_count', 'user.follower_count',
       'user.following_count', 'user.following_tag_count',
       'user.biography', 'user.external_url', 'user.total_igtv_videos',
       'user.total_clips_count', 'user.total_ar_effects',
       'user.usertags_count', 'user.is_favorite',
       'user.is_interest_account', 'user.has_chaining',
       'user.hd_profile_pic_versions[0].width',
       'user.hd_profile_pic_versions[0].height',
       'user.hd_profile_pic_versions[0].url',
       'user.hd_profile_pic_url_info.url',
       'user.hd_profile_pic_url_info.width',
       'user.hd_profile_pic_url_info.height',
       'user.mutual_followers_count', 'user.has_highlight_reels',
       'user.has_guides', 'user.can_be_reported_as_fr

In [5]:
user_profile.head()

input.session_id  input.user_id       user.pk  \
0  31997167011%3ALPek2vkom8Icfc%3A27     5932394085  5.932394e+09   
1  31997167011%3ALPek2vkom8Icfc%3A27     6898442222  6.898442e+09   
2  31997167011%3ALPek2vkom8Icfc%3A27     6905584184  6.905584e+09   
3  31997167011%3ALPek2vkom8Icfc%3A27     6924108892  6.924109e+09   
4  31997167011%3ALPek2vkom8Icfc%3A27     7064134243  7.064134e+09   

              user.username             user.full_name user.is_private  \
0             tammycburrows  Tammy W | yqr hairstylist           False   
1             mad.creationz                madi baines           False   
2  sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf           False   
3          ellanashairhive_                     Ellana           False   
4     envybeautysalonandspa    Envy Beauty Salon & Spa           False   

                                user.profile_pic_url  \
0  https://scontent.cdninstagram.com/v/t51.2885-1...   
1  https://scontent.cdninstagram.com/v/t51.2885-1...   
2  https://scontent.cdninstagram.com/v/t51.2885-1...   
3  https://scontent.cdninstagram.com/v/t51.2885-1...   
4  https://scontent.cdninstagram.com/v/t51.2885-1...   

              user.profile_pic_id user.is_verified  \
0  2414898732430991216_5932394085            False   
1  2373698590490927257_6898442222            False   
2  1690015909182536622_6905584184            False   
3  2329468451649630068_6924108892            False   
4  2215350723188003450_7064134243            False   

  user.has_anonymous_profile_picture  ...  user.smb_support_partner.url  \
0                              False  ...                           NaN   
1                              False  ...                           NaN   
2                              False  ...                           NaN   
3                              False  ...                           NaN   
4                              False  ...                           NaN   

   user.smb_support_partner.app_id  user.smb_support_partner.partner_name  \
0                              NaN                                    NaN   
1                              NaN                                    NaN   
2                              NaN                                    NaN   
3                              NaN                                    NaN   
4                              NaN                                    NaN   

   user.smb_support_partner.button_label  checkpoint.url checkpoint.lock  \
0                                    NaN             NaN             NaN   
1                                    NaN             NaN             NaN   
2                                    NaN             NaN             NaN   
3                                    NaN             NaN             NaN   
4                                    NaN             NaN             NaN   

  checkpoint.logout  checkpoint.challenge_api_path  checkpoint.show_rn_flow  \
0               NaN                            NaN                      NaN   
1               NaN                            NaN                      NaN   
2               NaN                            NaN                      NaN   
3               NaN                            NaN                      NaN   
4               NaN                            NaN                      NaN   

   checkpoint.flow_render_type  
0                          NaN  
1                          NaN  
2                          NaN  
3                          NaN  
4                          NaN  

[5 rows x 124 columns]

Thats a lot of fields! There are most likely a bunch which aren't needed for the sake of this assignment. Here are the ones I chose, and why:

1. username - Requirement for our output csv
2. full_name - Requirement for our output csv
3. biography - Might contain other information on type of account
4. account_type - Contains information about if they are business/pro
5. is_business/is_potential_business - Same as 4
6. city_name - Requirement for our output csv
7. public_email - Requirement for our output csv
8. zip - Requirement for our output csv
9. external_url - Requirement for our output csv

Let's drop the columns we don't neeed, and then take another look at our dataset.

In [6]:
columns = ['user.username','user.full_name','user.biography', 'user.account_type','user.is_business', 
           'user.is_potential_business', 'user.city_name', 'user.zip', 'user.external_url', 'user.public_email']
user_profile = user_profile.reindex(columns = columns)
user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                        NaN   
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography  user.account_type  \
0     Saskatchewan based Hairstylist for 30+ years!\...                2.0   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...                2.0   
2                            Sales Manager- Schwarzkopf                2.0   
3     Welcome hair honeys to my Instagram🍯  • Junior...                2.0   
4                 Where there is beauty, there is Envy.                2.0   
...                                                 ...                ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...                3.0   
3664  “ It’s nice to be important , but it’s more im...                1.0   
3665                                                NaN                1.0   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...                3.0   
3667                                                NaN                1.0   

     user.is_business user.is_potential_business        user.city_name  \
0                True                      False  Regina, Saskatchewan   
1                True                       True                   NaN   
2                True                      False                   NaN   
3                True                       True                   NaN   
4                True                       True      Toronto, Ontario   
...               ...                        ...                   ...   
3663            False                       True                   NaN   
3664            False                      False                   NaN   
3665            False                      False                   NaN   
3666            False                      False                   NaN   
3667            False                      False                   NaN   

     user.zip                        user.external_url  \
0         NaN                                      NaN   
1         NaN                                      NaN   
2         NaN  http://www.skpevents.co.uk/virtualeduk8   
3         NaN                                      NaN   
4     M9M 2X1                                      NaN   
...       ...                                      ...   
3663      NaN                                      NaN   
3664      NaN                                      NaN   
3665      NaN                                      NaN   
3666      NaN             https://tinyurl.com/yd65fzta   
3667      NaN                                      NaN   

             user.public_email  
0                          NaN  
1        madizmakeup@gmail.com  
2     sally.hawkins@henkel.com  
3        e.bobbyb123@gmail.com  
4                          NaN  
...                        ...  
3663                       NaN  
3664                       NaN  
3665                       NaN  
3666                       NaN  
3667                       NaN  

[3668 rows x 10 columns]

In [7]:
user_profile.loc[(user_profile['user.is_business'] == False) & (user_profile['user.is_potential_business'] == True)]

user.username                user.full_name  \
8              lakhan_____18                   ♛18___ʀᴀᴊᴀ♛   
11       mountain_hair_xpert              Danyel Schroeder   
25                   joyymia                      𝐌̃𝐢̃𝐚̃ ☾   
32          pclashspecialist                           NaN   
34    gigi.coloristaymake.up                GiGi_make up 💄   
...                      ...                           ...   
3624        euwesleyferreira               Wesley Ferreira   
3626        savannah.curbelo  Fort Worth Dallas Curly Hair   
3636           cyndidesigner               Cyndi's Designs   
3644             pamgillbrar                 Pam Gill Brar   
3663                 joyymia                      𝐌̃𝐢̃𝐚̃ ☾   

                                         user.biography  user.account_type  \
8     ♥️ ғᴀsʜɪᴏɴ ʙʟᴏɢɢᴇʀ 👕👖👟🕶️\n♥️ 🔹18/06/2001\n♥️ L...                1.0   
11    “Practice random kindness and senseless acts o...                1.0   
25    𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...                3.0   
32                                                  NaN                3.0   
34    🌺peluquería profesional \n🌺Taller de color ava...                1.0   
...                                                 ...                ...   
3624  @shoppingcachos\n▪︎ Agendamentos 19992128831\n...                3.0   
3626  Savannah Curbelo 🇨🇺\nHair Care Specialist 🌿💐@n...                3.0   
3636  Professional/Jewelry Artist || Poshmark: @cind...                1.0   
3644  Certified hair/makeup artist/hairdresser/esthe...                1.0   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...                3.0   

     user.is_business user.is_potential_business user.city_name user.zip  \
8               False                       True            NaN      NaN   
11              False                       True            NaN      NaN   
25              False                       True            NaN      NaN   
32              False                       True            NaN      NaN   
34              False                       True            NaN      NaN   
...               ...                        ...            ...      ...   
3624            False                       True            NaN      NaN   
3626            False                       True            NaN      NaN   
3636            False                       True            NaN      NaN   
3644            False                       True            NaN      NaN   
3663            False                       True            NaN      NaN   

                             user.external_url           user.public_email  
8                                          NaN                         NaN  
11                                         NaN                         NaN  
25                                         NaN                         NaN  
32                                         NaN                         NaN  
34                                         NaN                         NaN  
...                                        ...                         ...  
3624  http://www.youtube.com/c/WesleyFerreiraa      wesley_wdp@hotmail.com  
3626           http://www.savannahcurbelo.com/  savannah.curbelo@gmail.com  
3636                                       NaN                         NaN  
3644                                       NaN                         NaN  
3663                                       NaN                         NaN  

[288 rows x 10 columns]

Let's first determine if the account is a pro or a business. This method is not perfect due to the varience in account types and how they are defined per account.

Logic: If user.account_type == 1.0 or 3.0, and user.is_business == False, and user.is_potential_busienss == False, then the account is a personal account, and should be classified as Pro

If user.account_type == 2.0, and user.is_business == True, and user.is_potential_business == True, then the account is a business account, and should be classified as Business

If user.is_business == True, and user.is_potential_business == False, then the account is a business account, and should be classified as Pro

If user.is_business == False, and user.is_potential_business == True, then the account is a business account, and should be classified as Business

The reasoning behind this logic is that Instagram is more thorough with their algorithm in determining if the account is potentially a business (user.is_potential_business), and will be more accurate than any decision I come up with. 

Another approach which could be taken would be to manually label half of the data as Pro or Business, and create a simple machine learning model with scikit-learn, or a more advanced neural network to analyze vairous fields, and train it to sufficient accuracy. As there is not a training dataset availible for me to use, nor do I have the time or ressources to manually label them, I will not be creating a model.

I will now create a new column for account type (Pro or Business), and sort every account into it.

In [8]:
user_profile.insert(3, 'user.account', 0)

In [9]:
def account_condition(user_profile):
    #True -> Business, False -> Pro
    return_value = 'Pro'
    if user_profile['user.is_business'] == True and user_profile['user.is_potential_business'] == True and user_profile['user.account_type'] == 2.0:
        return_value = 'Business'
    elif user_profile['user.is_business'] == False and user_profile['user.is_potential_business'] == True:
        return_value = 'Business'
    return return_value

user_profile['user.account'] = user_profile.apply(account_condition, axis = 1)

user_profile.head()

user.username             user.full_name  \
0             tammycburrows  Tammy W | yqr hairstylist   
1             mad.creationz                madi baines   
2  sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3          ellanashairhive_                     Ellana   
4     envybeautysalonandspa    Envy Beauty Salon & Spa   

                                      user.biography user.account  \
0  Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1  🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                         Sales Manager- Schwarzkopf          Pro   
3  Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4              Where there is beauty, there is Envy.     Business   

   user.account_type user.is_business user.is_potential_business  \
0                2.0             True                      False   
1                2.0             True                       True   
2                2.0             True                      False   
3                2.0             True                       True   
4                2.0             True                       True   

         user.city_name user.zip                        user.external_url  \
0  Regina, Saskatchewan      NaN                                      NaN   
1                   NaN      NaN                                      NaN   
2                   NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                   NaN      NaN                                      NaN   
4      Toronto, Ontario  M9M 2X1                                      NaN   

          user.public_email  
0                       NaN  
1     madizmakeup@gmail.com  
2  sally.hawkins@henkel.com  
3     e.bobbyb123@gmail.com  
4                       NaN

I'll now drop the user.account_type, user.is_business, and user.is_potential_business fields as we have algamated it into user.account

In [10]:
user_profile = user_profile.drop(['user.account_type', 'user.is_business', 'user.is_potential_business'], axis = 1)
user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                        NaN   
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                            Sales Manager- Schwarzkopf          Pro   
3     Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4                 Where there is beauty, there is Envy.     Business   
...                                                 ...          ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...     Business   
3664  “ It’s nice to be important , but it’s more im...          Pro   
3665                                                NaN          Pro   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...          Pro   
3667                                                NaN          Pro   

            user.city_name user.zip                        user.external_url  \
0     Regina, Saskatchewan      NaN                                      NaN   
1                      NaN      NaN                                      NaN   
2                      NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                      NaN      NaN                                      NaN   
4         Toronto, Ontario  M9M 2X1                                      NaN   
...                    ...      ...                                      ...   
3663                   NaN      NaN                                      NaN   
3664                   NaN      NaN                                      NaN   
3665                   NaN      NaN                                      NaN   
3666                   NaN      NaN             https://tinyurl.com/yd65fzta   
3667                   NaN      NaN                                      NaN   

             user.public_email  
0                          NaN  
1        madizmakeup@gmail.com  
2     sally.hawkins@henkel.com  
3        e.bobbyb123@gmail.com  
4                          NaN  
...                        ...  
3663                       NaN  
3664                       NaN  
3665                       NaN  
3666                       NaN  
3667                       NaN  

[3668 rows x 8 columns]

I'll add a email field, and extract the email from the biography if applicable, and merge it with user.public_email.

In [11]:
user_profile.insert(7, 'user.email', 0)
user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                        NaN   
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                            Sales Manager- Schwarzkopf          Pro   
3     Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4                 Where there is beauty, there is Envy.     Business   
...                                                 ...          ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...     Business   
3664  “ It’s nice to be important , but it’s more im...          Pro   
3665                                                NaN          Pro   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...          Pro   
3667                                                NaN          Pro   

            user.city_name user.zip                        user.external_url  \
0     Regina, Saskatchewan      NaN                                      NaN   
1                      NaN      NaN                                      NaN   
2                      NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                      NaN      NaN                                      NaN   
4         Toronto, Ontario  M9M 2X1                                      NaN   
...                    ...      ...                                      ...   
3663                   NaN      NaN                                      NaN   
3664                   NaN      NaN                                      NaN   
3665                   NaN      NaN                                      NaN   
3666                   NaN      NaN             https://tinyurl.com/yd65fzta   
3667                   NaN      NaN                                      NaN   

      user.email         user.public_email  
0              0                       NaN  
1              0     madizmakeup@gmail.com  
2              0  sally.hawkins@henkel.com  
3              0     e.bobbyb123@gmail.com  
4              0                       NaN  
...          ...                       ...  
3663           0                       NaN  
3664           0                       NaN  
3665           0                       NaN  
3666           0                       NaN  
3667           0                       NaN  

[3668 rows x 9 columns]

In [12]:
def find_email(text):
    email = re.findall(r'[\w\.-]+@[\w\.-]+',str(text))
    return ",".join(email)
user_profile['user.email']= user_profile['user.biography'].apply(lambda x: find_email(x))
user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                        NaN   
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                            Sales Manager- Schwarzkopf          Pro   
3     Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4                 Where there is beauty, there is Envy.     Business   
...                                                 ...          ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...     Business   
3664  “ It’s nice to be important , but it’s more im...          Pro   
3665                                                NaN          Pro   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...          Pro   
3667                                                NaN          Pro   

            user.city_name user.zip                        user.external_url  \
0     Regina, Saskatchewan      NaN                                      NaN   
1                      NaN      NaN                                      NaN   
2                      NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                      NaN      NaN                                      NaN   
4         Toronto, Ontario  M9M 2X1                                      NaN   
...                    ...      ...                                      ...   
3663                   NaN      NaN                                      NaN   
3664                   NaN      NaN                                      NaN   
3665                   NaN      NaN                                      NaN   
3666                   NaN      NaN             https://tinyurl.com/yd65fzta   
3667                   NaN      NaN                                      NaN   

     user.email         user.public_email  
0                                     NaN  
1                   madizmakeup@gmail.com  
2                sally.hawkins@henkel.com  
3                   e.bobbyb123@gmail.com  
4                                     NaN  
...         ...                       ...  
3663                                  NaN  
3664                                  NaN  
3665                                  NaN  
3666                                  NaN  
3667                                  NaN  

[3668 rows x 9 columns]

In [13]:
#Verifying it did something
print(user_profile['user.email'].unique())

['' 'hairbyrhiannonv@gmail.com' 'gianluca.amatulli@icloud.com'
 'Professional@Paul' 'michelediniz127@gmail.com' '1kailani14u@gmail.com'
 'ninysteph@outlook.com' 'ronda@shearhair.ca' 'info@hairbyreema.com'
 'Graduated@schwarzkopfpro' 'rooyvicky@gmail.com' 'fionawills35@gmail.com'
 'personalmakeovercr@gmail.com' '𝚜𝚝𝚎𝚙𝚑𝚊𝚗𝚒𝚎.𝚙𝚊𝚝𝚘𝚒𝚗𝚎@𝚐𝚖𝚊𝚒𝚕.𝚌𝚘𝚖'
 '-@sevimli_shopping_baki' 'Monicalouisestyles@gmail.com'
 'Velvetfiddler@gmail.com' 'eusougracy@gmail.com'
 'fachionbeauty1996@gmail.com' '-@thakurparvesh99' 'll@hdUill'
 '28LeLocal@gmail.com' 'sylverlea@gmail.com' 'info@lafabriquecrepue.com'
 'Santamorelb@gmail.com' 'todos.@edisonlozano' 'piquetteceline@gmail.com'
 'avidehbakhshi@yahoo.com' 'melinamota@gmail.com'
 'giftedhandsacademytt@gmail.com' 'info@hairartisans.ca'
 'beautybycarlyd@gmail.com' 'info@erginsasmaz.com.tr'
 'rentals@chairmanmills.com' 'preetycuremua@gmail.com'
 'linamirmina@hotmail.com' 'lisadinhhairstudio@gmail.com'
 'danniefuller777@gmail.com' 'marcoocampoy@gmail.com' 'fb@nadiaSty

In [14]:
user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                        NaN   
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                            Sales Manager- Schwarzkopf          Pro   
3     Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4                 Where there is beauty, there is Envy.     Business   
...                                                 ...          ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...     Business   
3664  “ It’s nice to be important , but it’s more im...          Pro   
3665                                                NaN          Pro   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...          Pro   
3667                                                NaN          Pro   

            user.city_name user.zip                        user.external_url  \
0     Regina, Saskatchewan      NaN                                      NaN   
1                      NaN      NaN                                      NaN   
2                      NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                      NaN      NaN                                      NaN   
4         Toronto, Ontario  M9M 2X1                                      NaN   
...                    ...      ...                                      ...   
3663                   NaN      NaN                                      NaN   
3664                   NaN      NaN                                      NaN   
3665                   NaN      NaN                                      NaN   
3666                   NaN      NaN             https://tinyurl.com/yd65fzta   
3667                   NaN      NaN                                      NaN   

     user.email         user.public_email  
0                                     NaN  
1                   madizmakeup@gmail.com  
2                sally.hawkins@henkel.com  
3                   e.bobbyb123@gmail.com  
4                                     NaN  
...         ...                       ...  
3663                                  NaN  
3664                                  NaN  
3665                                  NaN  
3666                                  NaN  
3667                                  NaN  

[3668 rows x 9 columns]

In [15]:
#Merging the columns
user_profile['user.email'] = user_profile['user.email'] + " " + user_profile['user.public_email'] 

In [16]:
user_profile.head()

user.username             user.full_name  \
0             tammycburrows  Tammy W | yqr hairstylist   
1             mad.creationz                madi baines   
2  sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3          ellanashairhive_                     Ellana   
4     envybeautysalonandspa    Envy Beauty Salon & Spa   

                                      user.biography user.account  \
0  Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1  🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                         Sales Manager- Schwarzkopf          Pro   
3  Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4              Where there is beauty, there is Envy.     Business   

         user.city_name user.zip                        user.external_url  \
0  Regina, Saskatchewan      NaN                                      NaN   
1                   NaN      NaN                                      NaN   
2                   NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                   NaN      NaN                                      NaN   
4      Toronto, Ontario  M9M 2X1                                      NaN   

                  user.email         user.public_email  
0                        NaN                       NaN  
1      madizmakeup@gmail.com     madizmakeup@gmail.com  
2   sally.hawkins@henkel.com  sally.hawkins@henkel.com  
3      e.bobbyb123@gmail.com     e.bobbyb123@gmail.com  
4                        NaN                       NaN

In [17]:
user_profile = user_profile.drop(['user.public_email'], axis = 1)
user_profile.head()

user.username             user.full_name  \
0             tammycburrows  Tammy W | yqr hairstylist   
1             mad.creationz                madi baines   
2  sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3          ellanashairhive_                     Ellana   
4     envybeautysalonandspa    Envy Beauty Salon & Spa   

                                      user.biography user.account  \
0  Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1  🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                         Sales Manager- Schwarzkopf          Pro   
3  Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4              Where there is beauty, there is Envy.     Business   

         user.city_name user.zip                        user.external_url  \
0  Regina, Saskatchewan      NaN                                      NaN   
1                   NaN      NaN                                      NaN   
2                   NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                   NaN      NaN                                      NaN   
4      Toronto, Ontario  M9M 2X1                                      NaN   

                  user.email  
0                        NaN  
1      madizmakeup@gmail.com  
2   sally.hawkins@henkel.com  
3      e.bobbyb123@gmail.com  
4                        NaN

Looks like some emails apprear twice, so let's also clean that up.

In [18]:
user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                        NaN   
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                            Sales Manager- Schwarzkopf          Pro   
3     Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4                 Where there is beauty, there is Envy.     Business   
...                                                 ...          ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...     Business   
3664  “ It’s nice to be important , but it’s more im...          Pro   
3665                                                NaN          Pro   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...          Pro   
3667                                                NaN          Pro   

            user.city_name user.zip                        user.external_url  \
0     Regina, Saskatchewan      NaN                                      NaN   
1                      NaN      NaN                                      NaN   
2                      NaN      NaN  http://www.skpevents.co.uk/virtualeduk8   
3                      NaN      NaN                                      NaN   
4         Toronto, Ontario  M9M 2X1                                      NaN   
...                    ...      ...                                      ...   
3663                   NaN      NaN                                      NaN   
3664                   NaN      NaN                                      NaN   
3665                   NaN      NaN                                      NaN   
3666                   NaN      NaN             https://tinyurl.com/yd65fzta   
3667                   NaN      NaN                                      NaN   

                     user.email  
0                           NaN  
1         madizmakeup@gmail.com  
2      sally.hawkins@henkel.com  
3         e.bobbyb123@gmail.com  
4                           NaN  
...                         ...  
3663                        NaN  
3664                        NaN  
3665                        NaN  
3666                        NaN  
3667                        NaN  

[3668 rows x 8 columns]

I'll first extract emails, phone number and website from the bio , becase I will be removing all symbols after. I'll just use regex to do so.

In [19]:
#Extract email
user_profile['user.email1'] = user_profile['user.biography'].str.extract(r'([\w\.-]+@[\w\.-]+\.\w+)')

#Filling NaN so merging doesn't break the columns
user_profile = user_profile.fillna(' ')

#Merging
user_profile['user.email'] = user_profile['user.email1'] + ' ' + user_profile['user.email']
user_profile['user.email'] = user_profile['user.email'].str.split().str.get(0)

user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan based Hairstylist for 30+ years!\...          Pro   
1     🤍hair student in vancouver-ish bc\n🤍personal: ...     Business   
2                            Sales Manager- Schwarzkopf          Pro   
3     Welcome hair honeys to my Instagram🍯  • Junior...     Business   
4                 Where there is beauty, there is Envy.     Business   
...                                                 ...          ...   
3663  𝐖̃ 𝐢̃ 𝐭̃ 𝐜̃ 𝐡̃ 𝐲̃ ♡ 𝒻𝒾𝑔𝒽𝓉𝒾𝓃𝑔 𝑒𝓋𝒾𝓁 𝒷𝓎 𝓂𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉...     Business   
3664  “ It’s nice to be important , but it’s more im...          Pro   
3665                                                             Pro   
3666  ✨ accepting new clients ✨ \nBlondes&Balayage\n...          Pro   
3667                                                             Pro   

            user.city_name user.zip                        user.external_url  \
0     Regina, Saskatchewan                                                     
1                                                                              
2                                    http://www.skpevents.co.uk/virtualeduk8   
3                                                                              
4         Toronto, Ontario  M9M 2X1                                            
...                    ...      ...                                      ...   
3663                                                                           
3664                                                                           
3665                                                                           
3666                                            https://tinyurl.com/yd65fzta   
3667                                                                           

                    user.email user.email1  
0                          NaN              
1        madizmakeup@gmail.com              
2     sally.hawkins@henkel.com              
3        e.bobbyb123@gmail.com              
4                          NaN              
...                        ...         ...  
3663                       NaN              
3664                       NaN              
3665                       NaN              
3666                       NaN              
3667                       NaN              

[3668 rows x 9 columns]

In [20]:
#Dropping user.email1
user_profile = user_profile.drop(['user.email1'], axis = 1)

In [21]:
#Parsing website
user_profile['user.website'] = user_profile['user.biography'].str.extract(r'(https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+)')
user_profile['user.website'].unique()

#Filling NaN so merging doesn't break the columns
user_profile = user_profile.fillna(' ')

#Merging
user_profile['user.website'] = user_profile['user.website'] + ' ' + user_profile['user.external_url']
user_profile['user.website'] = user_profile['user.website'].str.split().str.get(0)

user_profile = user_profile.drop(['user.external_url'], axis = 1)
user_profile['user.website'].unique()

array([nan, 'http://www.skpevents.co.uk/virtualeduk8',
       'http://www.eylul.co.uk/', 'http://edgebarbers.com/',
       'https://www.facebook.com', 'https://www.facebook.com/1043951867',
       'http://www.yourspacesalons.ca/book-now/',
       'http://www.linktr.ee/bliinkstudio/',
       'https://tinyurl.com/yd65fzta', 'https://nese.kyani.net/gb/en-gb/',
       'http://taylored-hair.com/', 'http://headlinessalon.ca/james',
       'https://tellonym.me/martinezelijah192',
       'http://tigiprofessional.com/',
       'https://api.whatsapp.com/send?phone=5581994204281',
       'http://nazaninbeauty.studio/',
       'https://api.whatsapp.com/send?phone=5511957784398',
       'http://www.superhairextensions.com/', 'https://m.facebook.com/',
       'http://botaniquehair.com/', 'http://gmail.com/',
       'https://peluqueriacoquetas.tahe.es/coloracion/tinte-profesional-3/pack-tinte-oxigenada-guantes',
       'http://www.studiowsalon.ca/', 'http://www.upwardstyles.com/',
       'https://sal

In [22]:
#Parsing phone number
user_profile['user.phone'] = user_profile['user.biography'].str.extract(r'([\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9])')

#Filling NaN so merging doesn't break the columns
user_profile = user_profile.fillna(' ')

user_profile['user.phone'].unique()

array([' ', '208 8041020', '905 898 4464', '89283443203', '3816972837',
       '(647)4665567', '(949)450-0300', '11 9 47574912', '(317)742-7471',
       '+380995372360', '289-829-1004', '+31(0)647701878', '7454647162',
       '917-870-78-58', '552 710 82 42 - 0507 727 64 00', '+56949466760',
       '7710881743', '553 739 82 20', '10.4768.7764', '647-227-5283',
       '780-933-6362', '352-573-4428', '647 835 9724', '613-424-2600',
       '8291914949', '(403) 454-4556', '6476410266', '519-854-4707',
       '905 274 3500', '(651)-340-0766', '3482558359', '1-306-434-7855',
       '843-856-8500', '2127122800', '(403)764-3240', '306-649-2508',
       '9123133016', '1-306-683-222', '519-680-0129', '613-695-0789',
       '620 2222190', '705-719-7777', '780-986-3520', '8077779729',
       '519-974-5453', '+56 953232578', '780-986-7271', '3572-15506550',
       '5492616755915', '961 773303', '(559) 393-1671', '3348134976',
       '4492893261', '510.355.4122', '971-371-0946', '(21) 21976808018',


Lets use geograpy to extract countries/cities from the biography. Regex isn't as useful to use as the biography varies too much, so its much better to use a library dedicated to it :). We'll also do some cleaning to make it more accurate.

I'll also remove the stopwords, and convert any abbreviations for the states/provinces (not enough time for every country) to make it more accurate

In [23]:
#Fill all empty values so the function does not fail
user_profile = user_profile.fillna(' ') 
user_profile = user_profile.replace('nan', ' ')

#Using regex to remove all symbols, replacing them with a space for better searching
user_profile['user.biography'] = user_profile['user.biography'].replace('[^A-Za-z0-9]+',' ', regex = True)

#Capatilizing the first letter of all text for geograpy
user_profile['user.biography'] = user_profile['user.biography'].str.title()

user_profile

user.username             user.full_name  \
0                tammycburrows  Tammy W | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  Sally Hawkins-Schwarzkopf   
3             ellanashairhive_                     Ellana   
4        envybeautysalonandspa    Envy Beauty Salon & Spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              Sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     Alyssia |  Winnipeg MB   
3667             emily_j_grant                Emily Grant   

                                         user.biography user.account  \
0     Saskatchewan Based Hairstylist For 30 Years Ba...          Pro   
1      Hair Student In Vancouver Ish Bc Personal Mad...     Business   
2                             Sales Manager Schwarzkopf          Pro   
3     Welcome Hair Honeys To My Instagram Junior Hai...     Business   
4                  Where There Is Beauty There Is Envy      Business   
...                                                 ...          ...   
3663           Witchyaesthetic D O G M O M Cosmetology      Business   
3664   It S Nice To Be Important But It S More Impor...          Pro   
3665                                                             Pro   
3666   Accepting New Clients Blondes Balayage Prep C...          Pro   
3667                                                             Pro   

            user.city_name user.zip                user.email  \
0     Regina, Saskatchewan                                      
1                                       madizmakeup@gmail.com   
2                                    sally.hawkins@henkel.com   
3                                       e.bobbyb123@gmail.com   
4         Toronto, Ontario  M9M 2X1                             
...                    ...      ...                       ...   
3663                                                            
3664                                                            
3665                                                            
3666                                                            
3667                                                            

                                 user.website user.phone  
0                                                         
1                                                         
2     http://www.skpevents.co.uk/virtualeduk8             
3                                                         
4                                                         
...                                       ...        ...  
3663                                                      
3664                                                      
3665                                                      
3666             https://tinyurl.com/yd65fzta             
3667                                                      

[3668 rows x 9 columns]

Removing Stopwords using genshin, stopwords are defined in the first cell

In [24]:
user_profile = user_profile.apply(lambda x: x.astype(str).str.lower())

#Removing stopwords
for i in range (len(user_profile)):
    text = user_profile['user.biography'].iloc[i]
    text = remove_stopwords(text.lower())
    
    user_profile['user.biography'].iloc[i] = text


user_profile


user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0     saskatchewan based hairstylist 30 years barber...          pro   
1     hair student vancouver ish bc personal madiiba...     business   
2                             sales manager schwarzkopf          pro   
3     welcome hair honeys instagram junior hairstyli...     business   
4                                           beauty envy     business   
...                                                 ...          ...   
3663            witchyaesthetic d o g m o m cosmetology     business   
3664                  s nice important s important nice          pro   
3665                                                             pro   
3666  accepting new clients blondes balayage prep co...          pro   
3667                                                             pro   

            user.city_name user.zip                user.email  \
0     regina, saskatchewan                                      
1                                       madizmakeup@gmail.com   
2                                    sally.hawkins@henkel.com   
3                                       e.bobbyb123@gmail.com   
4         toronto, ontario  m9m 2x1                             
...                    ...      ...                       ...   
3663                                                            
3664                                                            
3665                                                            
3666                                                            
3667                                                            

                                 user.website user.phone  
0                                                         
1                                                         
2     http://www.skpevents.co.uk/virtualeduk8             
3                                                         
4                                                         
...                                       ...        ...  
3663                                                      
3664                                                      
3665                                                      
3666             https://tinyurl.com/yd65fzta             
3667                                                      

[3668 rows x 9 columns]

I was originally going to use nltk, however gensim had more stopwords, which may lead to a greater accuracy

In [25]:
"""

user_profile = user_profile.apply(lambda x: x.astype(str).str.lower())

#Removing stopwords
for i in range (len(user_profile)):
    text = user_profile['user.biography'].iloc[i]
    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    user_profile['user.biography'].iloc[i] = ' '.join(tokens_without_sw)


user_profile
"""

"\n\nuser_profile = user_profile.apply(lambda x: x.astype(str).str.lower())\n\n#Removing stopwords\nfor i in range (len(user_profile)):\n    text = user_profile['user.biography'].iloc[i]\n    text_tokens = word_tokenize(text)\n\n    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]\n    user_profile['user.biography'].iloc[i] = ' '.join(tokens_without_sw)\n\n\nuser_profile\n"

I'll now check for common abbreviates of canadian provinces and usa states, and adding Canada and United States to make it easier for the parser.

In [26]:
for i in range (len(user_profile)):
    text = " " + user_profile['user.biography'].iloc[i] + " "
    for word, initial in cp.can_province_names.items():
        text = text.upper().replace(' ' + word.upper() + ' ', ' Canada ' + initial + ' ').lower() #Abbrev to full name
        text = text.title().replace(' ' + initial.title() + ' ', ' Canada ' + initial + ' ') #Adding Canada if province found

    for word, initial in us.us_states_name.items():
        text = text.upper().replace(' ' + word.upper() + ' ', ' United States ' + initial + ' ').lower() #Abbrev to full name
        text = text.title().replace(' ' + initial.title() + ' ', ' United States ' + initial + ' ') #adding usa if state found 
        
    user_profile['user.biography'].iloc[i] = text

user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
2                            Sales Manager Schwarzkopf           pro   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                                          Beauty Envy      business   
...                                                 ...          ...   
3663           Witchyaesthetic D O G M O M Cosmetology      business   
3664                 S Nice Important S Important Nice           pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

            user.city_name user.zip                user.email  \
0     regina, saskatchewan                                      
1                                       madizmakeup@gmail.com   
2                                    sally.hawkins@henkel.com   
3                                       e.bobbyb123@gmail.com   
4         toronto, ontario  m9m 2x1                             
...                    ...      ...                       ...   
3663                                                            
3664                                                            
3665                                                            
3666                                                            
3667                                                            

                                 user.website user.phone  
0                                                         
1                                                         
2     http://www.skpevents.co.uk/virtualeduk8             
3                                                         
4                                                         
...                                       ...        ...  
3663                                                      
3664                                                      
3665                                                      
3666             https://tinyurl.com/yd65fzta             
3667                                                      

[3668 rows x 9 columns]

I'll add 2 new fields for what the geograpy library gives me for the location of the account based off their instagram bio.

In [27]:
user_profile.insert(6, 'user.geocity', 0)
user_profile.insert(7, 'user.geocountry', 0)

I'll add the user.city_name to the biography for increased accuracy

In [28]:
user_profile['user.biography'] = user_profile['user.biography'] + " " + user_profile['user.city_name']
user_profile.head()

user.username             user.full_name  \
0             tammycburrows  tammy w | yqr hairstylist   
1             mad.creationz                madi baines   
2  sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3          ellanashairhive_                     ellana   
4     envybeautysalonandspa    envy beauty salon & spa   

                                      user.biography user.account  \
0   Canada Saskatchewan Based Hairstylist 30 Year...          pro   
1   Hair Student Vancouver Ish Canada Canada Brit...     business   
2                       Sales Manager Schwarzkopf             pro   
3   Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                      Beauty Envy  toronto, ontario     business   

         user.city_name user.zip  user.geocity  user.geocountry  \
0  regina, saskatchewan                      0                0   
1                                            0                0   
2                                            0                0   
3                                            0                0   
4      toronto, ontario  m9m 2x1             0                0   

                 user.email                             user.website  \
0                                                                      
1     madizmakeup@gmail.com                                            
2  sally.hawkins@henkel.com  http://www.skpevents.co.uk/virtualeduk8   
3     e.bobbyb123@gmail.com                                            
4                                                                      

  user.phone  
0             
1             
2             
3             
4

Time to use geograpy to parse the location of the account!

In [29]:
user_profile = user_profile.applymap(str)
for i in range(int(len(user_profile))):
    geodata = geograpy.get_geoPlace_context(text = user_profile['user.biography'].iloc[i])
    if geodata.cities:
        user_profile['user.geocity'].iloc[i] = geodata.cities[0]
    if geodata.countries:
        user_profile['user.geocountry'].iloc[i] = geodata.countries[0]
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
2                          Sales Manager Schwarzkopf             pro   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                         Beauty Envy  toronto, ontario     business   
...                                                 ...          ...   
3663         Witchyaesthetic D O G M O M Cosmetology        business   
3664               S Nice Important S Important Nice             pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

            user.city_name user.zip user.geocity user.geocountry  \
0     regina, saskatchewan                Canada          Canada   
1                                              0               0   
2                                              0               0   
3                                              0               0   
4         toronto, ontario  m9m 2x1            0               0   
...                    ...      ...          ...             ...   
3663                                           0               0   
3664                                           0               0   
3665                                           0               0   
3666                                           0               0   
3667                                           0               0   

                    user.email                             user.website  \
0                                                                         
1        madizmakeup@gmail.com                                            
2     sally.hawkins@henkel.com  http://www.skpevents.co.uk/virtualeduk8   
3        e.bobbyb123@gmail.com                                            
4                                                                         
...                        ...                                      ...   
3663                                                                      
3664                                                                      
3665                                                                      
3666                                       https://tinyurl.com/yd65fzta   
3667                                                                      

     user.phone  
0                
1                
2                
3                
4                
...         ...  
3663             
3664             
3665             
3666             
3667             

[3668 rows x 11 columns]

In [30]:
#Checking the data after parsing
user_profile['user.geocity'].unique()

array(['Canada', '0', 'Shiraz', 'New York', 'Jordan', 'Mexico', 'Turku',
       'Island', 'Portage', 'Russell', 'Portland', 'Britton', 'Salta',
       'Long', 'Toronto', 'Colombia', 'Chisinau', 'Faith', 'California',
       'Los', 'Boston', 'Teresina', 'Sos', 'Cleveland', 'Auckland',
       'Line', 'Syria', 'Mississauga', 'Canadian'], dtype=object)

In [31]:
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
2                          Sales Manager Schwarzkopf             pro   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                         Beauty Envy  toronto, ontario     business   
...                                                 ...          ...   
3663         Witchyaesthetic D O G M O M Cosmetology        business   
3664               S Nice Important S Important Nice             pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

            user.city_name user.zip user.geocity user.geocountry  \
0     regina, saskatchewan                Canada          Canada   
1                                              0               0   
2                                              0               0   
3                                              0               0   
4         toronto, ontario  m9m 2x1            0               0   
...                    ...      ...          ...             ...   
3663                                           0               0   
3664                                           0               0   
3665                                           0               0   
3666                                           0               0   
3667                                           0               0   

                    user.email                             user.website  \
0                                                                         
1        madizmakeup@gmail.com                                            
2     sally.hawkins@henkel.com  http://www.skpevents.co.uk/virtualeduk8   
3        e.bobbyb123@gmail.com                                            
4                                                                         
...                        ...                                      ...   
3663                                                                      
3664                                                                      
3665                                                                      
3666                                       https://tinyurl.com/yd65fzta   
3667                                                                      

     user.phone  
0                
1                
2                
3                
4                
...         ...  
3663             
3664             
3665             
3666             
3667             

[3668 rows x 11 columns]

I'll now carry over the city from the user.city_name to the geocity (more accurate than the parser). There is no countries field, so I'll leave that to geograpy.

In [32]:
user_profile['user.geocity'] = user_profile['user.city_name'].str.split().str.get(0)

In [33]:
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
2                          Sales Manager Schwarzkopf             pro   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                         Beauty Envy  toronto, ontario     business   
...                                                 ...          ...   
3663         Witchyaesthetic D O G M O M Cosmetology        business   
3664               S Nice Important S Important Nice             pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

            user.city_name user.zip user.geocity user.geocountry  \
0     regina, saskatchewan               regina,          Canada   
1                                            NaN               0   
2                                            NaN               0   
3                                            NaN               0   
4         toronto, ontario  m9m 2x1     toronto,               0   
...                    ...      ...          ...             ...   
3663                                         NaN               0   
3664                                         NaN               0   
3665                                         NaN               0   
3666                                         NaN               0   
3667                                         NaN               0   

                    user.email                             user.website  \
0                                                                         
1        madizmakeup@gmail.com                                            
2     sally.hawkins@henkel.com  http://www.skpevents.co.uk/virtualeduk8   
3        e.bobbyb123@gmail.com                                            
4                                                                         
...                        ...                                      ...   
3663                                                                      
3664                                                                      
3665                                                                      
3666                                       https://tinyurl.com/yd65fzta   
3667                                                                      

     user.phone  
0                
1                
2                
3                
4                
...         ...  
3663             
3664             
3665             
3666             
3667             

[3668 rows x 11 columns]

I'm going to drop the user.city_name field now as it is merged with geocity

In [34]:
user_profile = user_profile.drop(['user.city_name'], axis = 1)
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account user.zip  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro            
1      Hair Student Vancouver Ish Canada Canada Brit...     business            
2                          Sales Manager Schwarzkopf             pro            
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business            
4                         Beauty Envy  toronto, ontario     business  m9m 2x1   
...                                                 ...          ...      ...   
3663         Witchyaesthetic D O G M O M Cosmetology        business            
3664               S Nice Important S Important Nice             pro            
3665                                                             pro            
3666   Accepting New Clients Blondes Balayage Prep C...          pro            
3667                                                             pro            

     user.geocity user.geocountry                user.email  \
0         regina,          Canada                             
1             NaN               0     madizmakeup@gmail.com   
2             NaN               0  sally.hawkins@henkel.com   
3             NaN               0     e.bobbyb123@gmail.com   
4        toronto,               0                             
...           ...             ...                       ...   
3663          NaN               0                             
3664          NaN               0                             
3665          NaN               0                             
3666          NaN               0                             
3667          NaN               0                             

                                 user.website user.phone  
0                                                         
1                                                         
2     http://www.skpevents.co.uk/virtualeduk8             
3                                                         
4                                                         
...                                       ...        ...  
3663                                                      
3664                                                      
3665                                                      
3666             https://tinyurl.com/yd65fzta             
3667                                                      

[3668 rows x 10 columns]

I'll then search the bio once again for postal code/zip code, and create new columns to add it to. I'll then merge it together into 1 column.

In [35]:
user_profile['user.zip_bio'] = user_profile['user.biography'].str.extract(r'(\d{5}\-?\d{0,4})')
user_profile['user.postal_bio'] = user_profile['user.biography'].str.extract(r'(\b(?!.{0,7}[DFIOQU])[A-VXY]\d[A-Z][^-\w\d]\d[A-Z]\d\b)')
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account user.zip  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro            
1      Hair Student Vancouver Ish Canada Canada Brit...     business            
2                          Sales Manager Schwarzkopf             pro            
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business            
4                         Beauty Envy  toronto, ontario     business  m9m 2x1   
...                                                 ...          ...      ...   
3663         Witchyaesthetic D O G M O M Cosmetology        business            
3664               S Nice Important S Important Nice             pro            
3665                                                             pro            
3666   Accepting New Clients Blondes Balayage Prep C...          pro            
3667                                                             pro            

     user.geocity user.geocountry                user.email  \
0         regina,          Canada                             
1             NaN               0     madizmakeup@gmail.com   
2             NaN               0  sally.hawkins@henkel.com   
3             NaN               0     e.bobbyb123@gmail.com   
4        toronto,               0                             
...           ...             ...                       ...   
3663          NaN               0                             
3664          NaN               0                             
3665          NaN               0                             
3666          NaN               0                             
3667          NaN               0                             

                                 user.website user.phone user.zip_bio  \
0                                                                 NaN   
1                                                                 NaN   
2     http://www.skpevents.co.uk/virtualeduk8                     NaN   
3                                                                 NaN   
4                                                                 NaN   
...                                       ...        ...          ...   
3663                                                              NaN   
3664                                                              NaN   
3665                                                              NaN   
3666             https://tinyurl.com/yd65fzta                     NaN   
3667                                                              NaN   

     user.postal_bio  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
...              ...  
3663             NaN  
3664             NaN  
3665             NaN  
3666             NaN  
3667             NaN  

[3668 rows x 12 columns]

In [36]:
user_profile = user_profile.fillna('')
user_profile['user.zip'] = user_profile['user.zip'] + ' ' + user_profile['user.zip_bio'] + ' ' + user_profile['user.postal_bio']
user_profile['user.zip'].unique()

array(['   ', 'm9m 2x1  ', 'en3 4dr 8041020 ', 't1j 0h7  ', '70124.0  ',
       'ml6 0aw  ', '  892834432 ', '  2325504 ', '05743  ', 'l4l9p1  ',
       '  4665567 ', '13250020  ', '125310  ', '92691  ', '  47574912 ',
       '  56957 ', '89611 380995372 ', '12200  ', 'l0s 1e3  ', 'v1l1s9  ',
       '6661rh 647701878 ', '  074546471 ', '15552  ', '160009  ',
       '98020  ', '78703  ', '77091  ', 'co2 7uy  ', '32803  ',
       '  569494667 ', '  077108817 ', 'hg1 1dh  ', '  255531149 ',
       '34000  ', '43300  ', '900 7333318 ', '34608  ', 'm4t2t5  ',
       '33143  ', '89102  ', '400078 829191494 ', 't2s 0k4  ',
       't2t 0e1  ', '  647641026 ', '7105  ', 'n5c2t4  ', '  10752 ',
       '  348255835 ', '  29464 ', '  10023 ', '29582  ', 'tn38 9er  ',
       't2e 7p4  ', 'e8k1h2  ', 's7w 0t2  ', 'k6v 6c3  ', 'h9j 3l8  ',
       '01960  ', '  091231330 ', '6065  ', 'm4c1j4  ', '57025120  ',
       'n6e1v4  ', 'm3c 1k6  ', '76210  ', '1172 2222190 ', 'r3n 0e3  ',
       '  65315728 '

In [37]:
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
1                mad.creationz                madi baines   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
3             ellanashairhive_                     ellana   
4        envybeautysalonandspa    envy beauty salon & spa   
...                        ...                        ...   
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
2                          Sales Manager Schwarzkopf             pro   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                         Beauty Envy  toronto, ontario     business   
...                                                 ...          ...   
3663         Witchyaesthetic D O G M O M Cosmetology        business   
3664               S Nice Important S Important Nice             pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

       user.zip user.geocity user.geocountry                user.email  \
0                    regina,          Canada                             
1                                          0     madizmakeup@gmail.com   
2                                          0  sally.hawkins@henkel.com   
3                                          0     e.bobbyb123@gmail.com   
4     m9m 2x1       toronto,               0                             
...         ...          ...             ...                       ...   
3663                                       0                             
3664                                       0                             
3665                                       0                             
3666                                       0                             
3667                                       0                             

                                 user.website user.phone user.zip_bio  \
0                                                                       
1                                                                       
2     http://www.skpevents.co.uk/virtualeduk8                           
3                                                                       
4                                                                       
...                                       ...        ...          ...   
3663                                                                    
3664                                                                    
3665                                                                    
3666             https://tinyurl.com/yd65fzta                           
3667                                                                    

     user.postal_bio  
0                     
1                     
2                     
3                     
4                     
...              ...  
3663                  
3664                  
3665                  
3666                  
3667                  

[3668 rows x 12 columns]

I'll now split the dataframe into Pros and Businesses and search their bio for the service they offer.

In [38]:
user_pro = user_profile[user_profile['user.account'] == 'pro']
user_buis = user_profile[user_profile['user.account'] == 'business']

I'll now use the SalonEverywhere csv to define the search terms I'll be looking for.

In [39]:
search_terms = pd.read_csv('SalonEverywhere_Synonyms.csv')
color = search_terms.loc[search_terms.category == 'color']
color

parent category                                            synonym
0       blonde    color                              ['blonde', 'blondes']
1        black    color                                          ['black']
2        brown    color  ['brown', 'browns', 'brownish', 'browning', 'b...
3         dark    color  ['dark', 'darkest', 'darker', 'darkskins', 'da...
4          red    color  ['red', 'redheads', 'reddish', 'redish', 'redh...
5      caramel    color                            ['caramel', 'caramelo']
6          ash    color                                    ['ash', 'ashy']
7        honey    color                                          ['honey']
8         blue    color                     ['blue', 'blueberry', 'blues']
9       purple    color                                         ['purple']
10        pink    color                                           ['pink']
11        grey    color                                ['grey', 'greying']
12       white    color                                          ['white']
13    burgundy    color                                       ['burgundy']
14        gold    color                                 ['gold', 'golden']
15        rose    color                               ['rose', 'rosepath']
16      silver    color                                         ['silver']
17   chocolate    color                        ['chocolate', 'chocolates']
18      auburn    color                                         ['auburn']
19    platinum    color                                       ['platinum']
20      copper    color                                         ['copper']
21  strawberry    color                                     ['strawberry']
22        warm    color                                           ['warm']
23     rainbow    color                                        ['rainbow']
24      bright    color                              ['bright', 'brights']
25    lemonade    color                                       ['lemonade']
26        fair    color                                           ['fair']

In [40]:
colors = []
for i in range(len(color)):
    colors += ast.literal_eval(color.synonym[i])
colors

['blonde',
 'blondes',
 'black',
 'brown',
 'browns',
 'brownish',
 'browning',
 'brownskin',
 'lightbrown',
 'dark',
 'darkest',
 'darker',
 'darkskins',
 'darkskin',
 'red',
 'redheads',
 'reddish',
 'redish',
 'redhead',
 'caramel',
 'caramelo',
 'ash',
 'ashy',
 'honey',
 'blue',
 'blueberry',
 'blues',
 'purple',
 'pink',
 'grey',
 'greying',
 'white',
 'burgundy',
 'gold',
 'golden',
 'rose',
 'rosepath',
 'silver',
 'chocolate',
 'chocolates',
 'auburn',
 'platinum',
 'copper',
 'strawberry',
 'warm',
 'rainbow',
 'bright',
 'brights',
 'lemonade',
 'fair']

In [41]:
search_terms = pd.read_csv('SalonEverywhere_Synonyms.csv')
search_terms = search_terms[search_terms.category == 'service'].reset_index()
search_terms = search_terms.drop(['index'], axis = 1)
search_terms

parent category                                            synonym
0         hair  service  ['hair', 'hairstyles', 'haircut', 'haircuts', ...
1       braids  service  ['braids', 'braid', 'braided', 'braiding', 'br...
2     balayage  service                                       ['balayage']
3   highlights  service  ['highlights', 'highlight', 'highlighter', 'hi...
4          box  service                          ['box', 'boxer', 'boxes']
5        weave  service                  ['weave', 'weaves', 'quickweave']
6   extensions  service                        ['extensions', 'extension']
7         perm  service  ['perm', 'permed', 'perms', 'permanent', 'perm...
8        ombre  service                      ['ombre', 'sombre', 'hombre']
9      haircut  service                            ['haircut', 'haircuts']
10   lowlights  service                                      ['lowlights']
11     tapered  service                                        ['tapered']

In [42]:
hair = ast.literal_eval(search_terms.synonym[0])
braids = ast.literal_eval(search_terms.synonym[1])
balayage = ast.literal_eval(search_terms.synonym[2])
highlights = ast.literal_eval(search_terms.synonym[3])
box = ast.literal_eval(search_terms.synonym[4])
weave = ast.literal_eval(search_terms.synonym[5])
extensions = ast.literal_eval(search_terms.synonym[6])
perm = ast.literal_eval(search_terms.synonym[7])
ombre = ast.literal_eval(search_terms.synonym[8])
haircut = ast.literal_eval(search_terms.synonym[9])
lowlights = ast.literal_eval(search_terms.synonym[10])
tapered = ast.literal_eval(search_terms.synonym[11])

hair_stylist = ['']
for i in range (len(search_terms)):
    hair_stylist += ast.literal_eval(search_terms.synonym[i])
    
nail_tech = ['nail', 'technician', 'tech']
esthetician = ['esthetician', 'beauty']
barber = ['haircut', 'haricuts','men', 'male', 'boy', 'mens', 'boys']
spa = ['spa', 'spas']

In [43]:
user_pro['user.hair_stylist']= user_pro['user.biography'].str.contains('|'.join(hair_stylist))
user_pro['user.nail_tech']= user_pro['user.biography'].str.contains('|'.join(nail_tech))
user_pro['user.esthetician']= user_pro['user.biography'].str.contains('|'.join(esthetician))
user_pro['user.barber']= user_pro['user.biography'].str.contains('|'.join(barber))

#If the user is none of the above, its 'other'
def isOther(row):
    if row['user.hair_stylist'] == False and row['user.nail_tech'] == False and row['user.esthetician'] == False and row['user.barber'] == False:
        return True
    else:
        return False

user_pro['user.other'] = user_pro.apply(isOther, axis = 1)

#Replacing True/False with Appropriate text
user_pro['user.hair_stylist']= user_pro['user.hair_stylist'].replace(True,'Hair Stylist')
user_pro['user.nail_tech']= user_pro['user.nail_tech'].replace(True,'Nail Technician')
user_pro['user.esthetician']= user_pro['user.esthetician'].replace(True,'Esthetician')
user_pro['user.barber']= user_pro['user.barber'].replace(True,'Barber')
user_pro['user.other']= user_pro['user.other'].replace(True,'Other')
user_pro

<ipython-input-43-70f6b60fc400>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_pro['user.hair_stylist']= user_pro['user.biography'].str.contains('|'.join(hair_stylist))
<ipython-input-43-70f6b60fc400>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_pro['user.nail_tech']= user_pro['user.biography'].str.contains('|'.join(nail_tech))
<ipython-input-43-70f6b60fc400>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
6          vaghela_bhavesh7799              surendr nagar   
7               edgebarbersyql               edge barbers   
10                  floyd__leo                      floyd   
...                        ...                        ...   
3661                    zoe88w                        zoe   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
2                          Sales Manager Schwarzkopf             pro   
6                                                                pro   
7      Evolution Barbering Offers Haircuts Beard Tri...          pro   
10                                                               pro   
...                                                 ...          ...   
3661                                                             pro   
3664               S Nice Important S Important Nice             pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

       user.zip user.geocity user.geocountry                user.email  \
0                    regina,          Canada                             
2                                          0  sally.hawkins@henkel.com   
6                                          0                             
7     t1j 0h7    lethbridge,               0                             
10                                         0                             
...         ...          ...             ...                       ...   
3661                                       0                             
3664                                       0                             
3665                                       0                             
3666                                       0                             
3667                                       0                             

                                 user.website user.phone user.zip_bio  \
0                                                                       
2     http://www.skpevents.co.uk/virtualeduk8                           
6                                                                       
7                     http://edgebarbers.com/                           
10                                                                      
...                                       ...        ...          ...   
3661                                                                    
3664                                                                    
3665                                                                    
3666             https://tinyurl.com/yd65fzta                           
3667                                                                    

     user.postal_bio user.hair_stylist  user.nail_tech user.esthetician  \
0                         Hair Stylist           False            False   
2                         Hair Stylist           False            False   
6                         Hair Stylist           False            False   
7                         Hair Stylist           False            False   
10                        Hair Stylist           False            False   
...              ...               ...             ...              ...   
3661                      Hair Stylist           False            False   
3664                      Hair Stylist           Fals

Time to merge the cells!

In [44]:
user_pro = user_pro.replace(False, '')
user_pro = user_pro.applymap(str)

#Merge
user_pro['user.pro_type'] = user_pro['user.hair_stylist'] + ' ' + user_pro['user.nail_tech'] + ' ' + user_pro['user.esthetician'] + ' ' + user_pro['user.barber'] + ' ' + user_pro['user.other']

#Drop

user_pro = user_pro.drop(['user.hair_stylist', 'user.nail_tech', 'user.esthetician', 'user.barber', 'user.other'], axis = 1)
user_pro


user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
6          vaghela_bhavesh7799              surendr nagar   
7               edgebarbersyql               edge barbers   
10                  floyd__leo                      floyd   
...                        ...                        ...   
3661                    zoe88w                        zoe   
3664                sweet_sb05              sonia 🇱🇧 / 🇨🇦   
3665                  hirtho79                              
3666              like_alyssia     alyssia |  winnipeg mb   
3667             emily_j_grant                emily grant   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
2                          Sales Manager Schwarzkopf             pro   
6                                                                pro   
7      Evolution Barbering Offers Haircuts Beard Tri...          pro   
10                                                               pro   
...                                                 ...          ...   
3661                                                             pro   
3664               S Nice Important S Important Nice             pro   
3665                                                             pro   
3666   Accepting New Clients Blondes Balayage Prep C...          pro   
3667                                                             pro   

       user.zip user.geocity user.geocountry                user.email  \
0                    regina,          Canada                             
2                                          0  sally.hawkins@henkel.com   
6                                          0                             
7     t1j 0h7    lethbridge,               0                             
10                                         0                             
...         ...          ...             ...                       ...   
3661                                       0                             
3664                                       0                             
3665                                       0                             
3666                                       0                             
3667                                       0                             

                                 user.website user.phone user.zip_bio  \
0                                                                       
2     http://www.skpevents.co.uk/virtualeduk8                           
6                                                                       
7                     http://edgebarbers.com/                           
10                                                                      
...                                       ...        ...          ...   
3661                                                                    
3664                                                                    
3665                                                                    
3666             https://tinyurl.com/yd65fzta                           
3667                                                                    

     user.postal_bio     user.pro_type  
0                     Hair Stylist      
2                     Hair Stylist      
6                     Hair Stylist      
7                     Hair Stylist      
10                    Hair Stylist      
...              ...               ...  
3661                  Hair Stylist      
3664                  Hair Stylist      
3665                  Hair Stylist      
3666                  Hair Stylist      
3667                  Hair Stylist      

[2566 rows x 13 columns]

The businesses follow something similar:

In [45]:
user_buis['user.hair_salon']= user_buis['user.biography'].str.contains('|'.join(hair_stylist))
user_buis['user.nail_salon']= user_buis['user.biography'].str.contains('|'.join(nail_tech))
user_buis['user.spa']= user_buis['user.biography'].str.contains('|'.join(spa))
user_buis['user.barbershop']= user_buis['user.biography'].str.contains('|'.join(barber))


#Replacing True/False with Appropriate text
user_buis['user.hair_salon']= user_buis['user.hair_salon'].replace(True,'Hair Stylist')
user_buis['user.nail_salon']= user_buis['user.nail_salon'].replace(True,'Nail Salon')
user_buis['user.esthetician']= user_buis['user.spa'].replace(True,'Spa')
user_buis['user.barbershop']= user_buis['user.barbershop'].replace(True,'Barbershop')

user_buis

<ipython-input-45-984dab03c144>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_buis['user.hair_salon']= user_buis['user.biography'].str.contains('|'.join(hair_stylist))
<ipython-input-45-984dab03c144>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_buis['user.nail_salon']= user_buis['user.biography'].str.contains('|'.join(nail_tech))
<ipython-input-45-984dab03c144>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

user.username               user.full_name  \
1               mad.creationz                  madi baines   
3            ellanashairhive_                       ellana   
4       envybeautysalonandspa      envy beauty salon & spa   
5          eylulhairandbeauty  𝑬𝒀𝑳𝐔𝑳 𝑯𝑨𝑰𝑹 & 𝑩𝑬𝑨𝑼𝑻𝒀 𝑺𝑨𝑳𝑶𝑵 🧿   
8               lakhan_____18                  ♛18___ʀᴀᴊᴀ♛   
...                       ...                          ...   
3649                  v_iulia                юля воробьева   
3651         hairbyabbyingram                abby ingram✂️   
3652  valeska_contreras_carla     valeska  contreras rojas   
3662                  __tiale                                
3663                  joyymia                     𝐌̃𝐢̃𝐚̃ ☾   

                                         user.biography user.account  \
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                         Beauty Envy  toronto, ontario     business   
5      Eylulnails Eylulbeautyclinic Enquiries Appoin...     business   
8                           S 18 06 2001 L Car Lover        business   
...                                                 ...          ...   
3649                                                        business   
3651   Hairstylist Salonchezpierre John Casablanca S...     business   
3652       Estilista Profesional Mam Feliz Mis Bebes        business   
3662   Curated Spiritual Wisdom Greater Feeling Love        business   
3663         Witchyaesthetic D O G M O M Cosmetology        business   

              user.zip user.geocity user.geocountry  \
1                                                 0   
3                                                 0   
4            m9m 2x1       toronto,               0   
5     en3 4dr 8041020       london,               0   
8                                                 0   
...                ...          ...             ...   
3649                                              0   
3651         v1m 2y5       langley,               0   
3652                                              0   
3662                                              0   
3663                                              0   

                               user.email  \
1                   madizmakeup@gmail.com   
3                   e.bobbyb123@gmail.com   
4                                           
5                       hello@eylul.co.uk   
8                                           
...                                   ...   
3649              lerasonaanna@icloud.com   
3651             abbyingram98@hotmail.com   
3652  contrerasrojacarlavaleska@gmail.com   
3662                                        
3663                                        

                                      user.website   user.phone user.zip_bio  \
1                                                                              
3                                                                              
4                                                                              
5                          http://www.eylul.co.uk/  208 8041020      8041020   
8                                                                              
...                                            ...          ...          ...   
3649                                                                           
3651  https://www.vagaro.com/us02/salonchezpierre/                             
3652                                                                           
3662            http://www.linktr.ee/bliinkstudio/                             
3663                                                                           

     user.postal_bio user.hair_salon user.nail_salon  user.spa  \
1                       Hair Stylist           False     False   
3                       Hair Stylist           False     False   
4                       Hair Stylist         

In [46]:
user_buis = user_buis.replace(False, '')
user_buis = user_buis.applymap(str)

#Merge
user_buis['user.buis_type'] = user_buis['user.hair_salon'] + ' ' + user_buis['user.nail_salon'] + ' ' + user_buis['user.spa'] + ' ' + user_buis['user.barbershop']

#Drop
user_buis = user_buis.drop(['user.hair_salon', 'user.nail_salon', 'user.spa', 'user.barbershop', 'user.esthetician'], axis = 1)
user_buis 

user.username               user.full_name  \
1               mad.creationz                  madi baines   
3            ellanashairhive_                       ellana   
4       envybeautysalonandspa      envy beauty salon & spa   
5          eylulhairandbeauty  𝑬𝒀𝑳𝐔𝑳 𝑯𝑨𝑰𝑹 & 𝑩𝑬𝑨𝑼𝑻𝒀 𝑺𝑨𝑳𝑶𝑵 🧿   
8               lakhan_____18                  ♛18___ʀᴀᴊᴀ♛   
...                       ...                          ...   
3649                  v_iulia                юля воробьева   
3651         hairbyabbyingram                abby ingram✂️   
3652  valeska_contreras_carla     valeska  contreras rojas   
3662                  __tiale                                
3663                  joyymia                     𝐌̃𝐢̃𝐚̃ ☾   

                                         user.biography user.account  \
1      Hair Student Vancouver Ish Canada Canada Brit...     business   
3      Welcome Hair Honeys Instagram Junior Hairstyl...     business   
4                         Beauty Envy  toronto, ontario     business   
5      Eylulnails Eylulbeautyclinic Enquiries Appoin...     business   
8                           S 18 06 2001 L Car Lover        business   
...                                                 ...          ...   
3649                                                        business   
3651   Hairstylist Salonchezpierre John Casablanca S...     business   
3652       Estilista Profesional Mam Feliz Mis Bebes        business   
3662   Curated Spiritual Wisdom Greater Feeling Love        business   
3663         Witchyaesthetic D O G M O M Cosmetology        business   

              user.zip user.geocity user.geocountry  \
1                                                 0   
3                                                 0   
4            m9m 2x1       toronto,               0   
5     en3 4dr 8041020       london,               0   
8                                                 0   
...                ...          ...             ...   
3649                                              0   
3651         v1m 2y5       langley,               0   
3652                                              0   
3662                                              0   
3663                                              0   

                               user.email  \
1                   madizmakeup@gmail.com   
3                   e.bobbyb123@gmail.com   
4                                           
5                       hello@eylul.co.uk   
8                                           
...                                   ...   
3649              lerasonaanna@icloud.com   
3651             abbyingram98@hotmail.com   
3652  contrerasrojacarlavaleska@gmail.com   
3662                                        
3663                                        

                                      user.website   user.phone user.zip_bio  \
1                                                                              
3                                                                              
4                                                                              
5                          http://www.eylul.co.uk/  208 8041020      8041020   
8                                                                              
...                                            ...          ...          ...   
3649                                                                           
3651  https://www.vagaro.com/us02/salonchezpierre/                             
3652                                                                           
3662            http://www.linktr.ee/bliinkstudio/                             
3663                                                                           

     user.postal_bio                       user.buis_type  
1                                         Hair Stylist     
3                                         Hair Stylist     
4                                         Hair Stylist     
5  

Let's now look at the second csv file we have to determine the services each instagram user offers.

In [47]:
user_post.columns.values

array(['input.session_id', 'input.username', 'input.max_id',
       'items.taken_at', 'items.pk', 'items.id', 'items.device_timestamp',
       'items.media_type', 'items.code', 'items.client_cache_key',
       'items.filter_type', 'items.location.pk', 'items.location.name',
       'items.location.address', 'items.location.city',
       'items.location.short_name', 'items.location.lng',
       'items.location.lat', 'items.location.external_source',
       'items.location.facebook_places_id', 'items.lat', 'items.lng',
       'items.should_request_ads', 'items.user.pk', 'items.user.username',
       'items.user.full_name', 'items.user.is_private',
       'items.user.profile_pic_url', 'items.user.profile_pic_id',
       'items.user.is_verified',
       'items.user.has_anonymous_profile_picture',
       'items.user.is_unpublished', 'items.user.is_favorite',
       'items.user.latest_reel_media', 'items.can_viewer_reshare',
       'items.caption_is_edited', 'items.comment_likes_enabled',
   

We're interested in the content that they posted, as it might lead us to the services they offer.

In [48]:
user_post = user_post.reindex(columns = ['input.username','items.caption.text'])
user_post

input.username                                 items.caption.text
0       hairbypetrina  𝙰𝚋𝚜𝚘𝚞𝚕𝚝𝚎 - 𝚑𝚊𝚒𝚛 - 𝚝𝚛𝚊𝚗𝚜𝚏𝚘𝚛𝚖𝚝𝚒𝚘𝚗 \n\n𝚂𝚎𝚛𝚟𝚒𝚌𝚎: B...
1       hairbypetrina  Ａｂｓｏｌｕｔｅ － Ｂａｌａｙａｇｅ － ｇｏａｌｓ \n\nＨａｉｒ － ｂｙ － ｐｅ...
2       hairbypetrina  Ｔｒａｎｓｆｏｒｍａｔｉｏｎ － ｏｆ － ｔｈｅ － ｄａｙ \n\nＨａｉｒ -ｂｙ -...
3       hairbypetrina  Had the absolute pleasure to cut and colour th...
4       hairbypetrina  𝑯𝒂𝒅 𝒕𝒉𝒆 𝒃𝒆𝒔𝒕 𝒏𝒊𝒈𝒉𝒕 𝒘𝒊𝒕𝒉 𝒕𝒉𝒆 𝒃𝒆𝒔𝒕 𝒈𝒊𝒓𝒍𝒔 \n\n𝑭𝒐𝒐...
...               ...                                                ...
549656    haircandymd  Lots of regrowth but we did a bunch of babylig...
549657    haircandymd  Look how pretty and dimensional this is! Had t...
549658    haircandymd                      5.5 hour transformation! 🌈🌈🌈🌈
549659    haircandymd  Mermaids are real thou🧜‍♀️🧜‍♀️ #amirite #merma...
549660    haircandymd  This beautiful sunkissed look brought to you b...

[549661 rows x 2 columns]

I'll then group the posts by the username to make it easier and faster to search.

In [49]:
user_post = pd.read_csv('user_posts_v1.csv')

C:\Users\Quinn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,7,10,11,14,16,17,20,22,23,29,30,31,32,34,35,36,37,38,40,49,50,61,88,90,91,92,103,104,105,107,108,109,110,111,112,116,119,120,123,124,128,132,133,135,136,138,139,140,141,142,143,145,150,154,155,157,158,160,161,162,163,164,165,167,171,172,173,174,175,176,178,179,181,182,183,184,185,186,188,189,191,192,193,195,196,197,198,199,201,205,206,208,209,210,211,212,223,224,227,228,229,230,231,232,233,238,241,245,247,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [50]:
user_post = user_post.applymap(str)
user_post = user_post.groupby('input.username').agg({'items.caption.text':' '.join})
user_post = user_post.drop(['1.0'], axis = 0)
user_post

items.caption.text
input.username                                                   
hairbypetrina   𝙰𝚋𝚜𝚘𝚞𝚕𝚝𝚎 - 𝚑𝚊𝚒𝚛 - 𝚝𝚛𝚊𝚗𝚜𝚏𝚘𝚛𝚖𝚝𝚒𝚘𝚗 \n\n𝚂𝚎𝚛𝚟𝚒𝚌𝚎: B...
hairbypeytonxx  Fun little thing we did today 🥰 Fun color and ...
hairbypinkyd    🗣NEW SERVICE ANNOUNCEMENT!!! I have GlossMe by...
hairbypizer     Getting back to my roots with lots of vivids l...
hairbypmanning  Root drooooop ⬇️ #MONEYCHOP #yeg #yeghair #why...
...                                                           ...
store6546       nan nan Equipment sale! nan 👀COMING SOON... WO...
store_love83    ‏أنت لا تعلم في أي طريق \nيختبئ الفرح \nولك...
stormiest       #BobbyWayne is double digits! #10yearold #cele...
story5212                                                 nan nan
stpaul61        Before and After. It my Dyson girl Looking pre...

[1169 rows x 1 columns]

I'll now remove the stopwords before I go parsing the data and searching around.

In [51]:
user_post = user_post.fillna(' ') 
user_post = user_post.replace('nan', ' ')

#Using regex to remove all symbols, replacing them with a space for better searching
user_post['items.caption.text'] = user_post['items.caption.text'].replace('[^A-Za-z0-9]+',' ', regex = True)

user_post = user_post.apply(lambda x: x.astype(str).str.lower())

#Removing stopwords
for i in range (len(user_post)):
    text = user_post['items.caption.text'].iloc[i]
    text = remove_stopwords(text.lower())
    
    user_post['items.caption.text'].iloc[i] = text


user_post

items.caption.text
input.username                                                   
hairbypetrina   b beckyjanadehairandbeauty hairbypetrina hairg...
hairbypeytonxx  fun little thing today fun color cut today thx...
hairbypinkyd    new service announcement glossme designmehair ...
hairbypizer     getting roots lots vivids lately feel like wor...
hairbypmanning  root drooooop moneychop yeg yeghair whyteave s...
...                                                           ...
store6546       nan nan equipment sale nan coming soon wow hai...
store_love83    nan nan nan nan nan nan nan nan nan nan nan na...
stormiest       bobbywayne double digits 10yearold celebrate b...
story5212                                                 nan nan
stpaul61        dyson girl looking pretty cool boy lol taking ...

[1169 rows x 1 columns]

In [52]:
user_post['women cut'] = user_post['items.caption.text'].str.contains('|'.join(hair_stylist))
user_post['men cut'] = user_post['items.caption.text'].str.contains('|'.join(barber))
user_post['hair color'] = user_post['items.caption.text'].str.contains('|'.join(color))
user_post['hair extension'] = user_post['items.caption.text'].str.contains('|'.join(extensions))
user_post['balayage'] = user_post['items.caption.text'].str.contains('|'.join(balayage))
user_post['highlights'] = user_post['items.caption.text'].str.contains('|'.join(highlights))

user_post['women cut'] = user_post['women cut'].replace(True, "Women's Haircut")
user_post['men cut'] = user_post['men cut'].replace(True, "Men's Haircut")
user_post['hair color'] = user_post['hair color'].replace(True, "Hair Color")
user_post['hair extension'] = user_post['hair extension'].replace(True, "Hair Extensions")
user_post['balayage'] = user_post['balayage'].replace(True, "Balayage")
user_post['highlights'] = user_post['highlights'].replace(True, "Highlights")

#remove False Values
user_post = user_post.replace(False,'')
user_post = user_post.applymap(str)

user_post

items.caption.text  \
input.username                                                      
hairbypetrina   b beckyjanadehairandbeauty hairbypetrina hairg...   
hairbypeytonxx  fun little thing today fun color cut today thx...   
hairbypinkyd    new service announcement glossme designmehair ...   
hairbypizer     getting roots lots vivids lately feel like wor...   
hairbypmanning  root drooooop moneychop yeg yeghair whyteave s...   
...                                                           ...   
store6546       nan nan equipment sale nan coming soon wow hai...   
store_love83    nan nan nan nan nan nan nan nan nan nan nan na...   
stormiest       bobbywayne double digits 10yearold celebrate b...   
story5212                                                 nan nan   
stpaul61        dyson girl looking pretty cool boy lol taking ...   

                      women cut        men cut  hair color   hair extension  \
input.username                                                                
hairbypetrina   Women's Haircut  Men's Haircut              Hair Extensions   
hairbypeytonxx  Women's Haircut  Men's Haircut              Hair Extensions   
hairbypinkyd    Women's Haircut  Men's Haircut              Hair Extensions   
hairbypizer     Women's Haircut  Men's Haircut  Hair Color  Hair Extensions   
hairbypmanning  Women's Haircut  Men's Haircut                                
...                         ...            ...         ...              ...   
store6546       Women's Haircut  Men's Haircut                                
store_love83    Women's Haircut                                               
stormiest       Women's Haircut  Men's Haircut  Hair Color  Hair Extensions   
story5212       Women's Haircut                                               
stpaul61        Women's Haircut  Men's Haircut                                

                balayage  highlights  
input.username                        
hairbypetrina   Balayage  Highlights  
hairbypeytonxx  Balayage  Highlights  
hairbypinkyd    Balayage  Highlights  
hairbypizer     Balayage  Highlights  
hairbypmanning  Balayage              
...                  ...         ...  
store6546                             
store_love83                          
stormiest       Balayage  Highlights  
story5212                             
stpaul61        Balayage              

[1169 rows x 7 columns]

Time to merge into a column called services.

In [53]:
#Merge
user_post['user.service'] = user_post['women cut'] + ', ' + user_post['men cut'] + ', ' + user_post['hair color'] + ', ' + user_post['hair extension'] + ', ' + user_post['balayage'] + ', ' + user_post['highlights']

#Drop
user_post = user_post.drop(['women cut', 'men cut', 'hair color', 'hair extension', 'balayage', 'highlights'],axis = 1)

user_post

items.caption.text  \
input.username                                                      
hairbypetrina   b beckyjanadehairandbeauty hairbypetrina hairg...   
hairbypeytonxx  fun little thing today fun color cut today thx...   
hairbypinkyd    new service announcement glossme designmehair ...   
hairbypizer     getting roots lots vivids lately feel like wor...   
hairbypmanning  root drooooop moneychop yeg yeghair whyteave s...   
...                                                           ...   
store6546       nan nan equipment sale nan coming soon wow hai...   
store_love83    nan nan nan nan nan nan nan nan nan nan nan na...   
stormiest       bobbywayne double digits 10yearold celebrate b...   
story5212                                                 nan nan   
stpaul61        dyson girl looking pretty cool boy lol taking ...   

                                                     user.service  
input.username                                                     
hairbypetrina   Women's Haircut, Men's Haircut, , Hair Extensi...  
hairbypeytonxx  Women's Haircut, Men's Haircut, , Hair Extensi...  
hairbypinkyd    Women's Haircut, Men's Haircut, , Hair Extensi...  
hairbypizer     Women's Haircut, Men's Haircut, Hair Color, Ha...  
hairbypmanning     Women's Haircut, Men's Haircut, , , Balayage,   
...                                                           ...  
store6546                  Women's Haircut, Men's Haircut, , , ,   
store_love83                            Women's Haircut, , , , ,   
stormiest       Women's Haircut, Men's Haircut, Hair Color, Ha...  
story5212                               Women's Haircut, , , , ,   
stpaul61           Women's Haircut, Men's Haircut, , , Balayage,   

[1169 rows x 2 columns]

Let's do the same for user_profile as well! I'll merge pro and business back first before doing anything.

In [66]:
user_profile = pd.concat([user_pro, user_buis])
user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
6          vaghela_bhavesh7799              surendr nagar   
7               edgebarbersyql               edge barbers   
10                  floyd__leo                      floyd   
...                        ...                        ...   
3649                   v_iulia              юля воробьева   
3651          hairbyabbyingram              abby ingram✂️   
3652   valeska_contreras_carla   valeska  contreras rojas   
3662                   __tiale                              
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
2                          Sales Manager Schwarzkopf             pro   
6                                                                pro   
7      Evolution Barbering Offers Haircuts Beard Tri...          pro   
10                                                               pro   
...                                                 ...          ...   
3649                                                        business   
3651   Hairstylist Salonchezpierre John Casablanca S...     business   
3652       Estilista Profesional Mam Feliz Mis Bebes        business   
3662   Curated Spiritual Wisdom Greater Feeling Love        business   
3663         Witchyaesthetic D O G M O M Cosmetology        business   

       user.zip user.geocity user.geocountry  \
0                    regina,          Canada   
2                                          0   
6                                          0   
7     t1j 0h7    lethbridge,               0   
10                                         0   
...         ...          ...             ...   
3649                                       0   
3651  v1m 2y5       langley,               0   
3652                                       0   
3662                                       0   
3663                                       0   

                               user.email  \
0                                           
2                sally.hawkins@henkel.com   
6                                           
7                                           
10                                          
...                                   ...   
3649              lerasonaanna@icloud.com   
3651             abbyingram98@hotmail.com   
3652  contrerasrojacarlavaleska@gmail.com   
3662                                        
3663                                        

                                      user.website user.phone user.zip_bio  \
0                                                                            
2          http://www.skpevents.co.uk/virtualeduk8                           
6                                                                            
7                          http://edgebarbers.com/                           
10                                                                           
...                                            ...        ...          ...   
3649                                                                         
3651  https://www.vagaro.com/us02/salonchezpierre/                           
3652                                                                         
3662            http://www.linktr.ee/bliinkstudio/                           
3663                                                                         

     user.postal_bio     user.pro_type   user.buis_type  
0                     Hair Stylist                  NaN  
2                     Hair Stylist                  NaN  
6                     Hair Stylist                  NaN  
7                     Hair Stylist                  NaN  
10                    Hair Stylist                  NaN  
...             

In [67]:
user_profile['women cut'] = user_profile['user.biography'].str.contains('|'.join(hair_stylist))
user_profile['men cut'] = user_profile['user.biography'].str.contains('|'.join(barber))
user_profile['hair color'] = user_profile['user.biography'].str.contains('|'.join(color))
user_profile['hair extension'] = user_profile['user.biography'].str.contains('|'.join(extensions))
user_profile['balayage'] = user_profile['user.biography'].str.contains('|'.join(balayage))
user_profile['highlights'] = user_profile['user.biography'].str.contains('|'.join(highlights))

user_profile['women cut'] = user_profile['women cut'].replace(True, "Women's Haircut")
user_profile['men cut'] = user_profile['men cut'].replace(True, "Men's Haircut")
user_profile['hair color'] = user_profile['hair color'].replace(True, "Hair Color")
user_profile['hair extension'] = user_profile['hair extension'].replace(True, "Hair Extensions")
user_profile['balayage'] = user_profile['balayage'].replace(True, "Balayage")
user_profile['highlights'] = user_profile['highlights'].replace(True, "Highlights")

#remove False Values
user_profile = user_profile.replace(False,'')
user_profile = user_profile.applymap(str)

user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
6          vaghela_bhavesh7799              surendr nagar   
7               edgebarbersyql               edge barbers   
10                  floyd__leo                      floyd   
...                        ...                        ...   
3649                   v_iulia              юля воробьева   
3651          hairbyabbyingram              abby ingram✂️   
3652   valeska_contreras_carla   valeska  contreras rojas   
3662                   __tiale                              
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
2                          Sales Manager Schwarzkopf             pro   
6                                                                pro   
7      Evolution Barbering Offers Haircuts Beard Tri...          pro   
10                                                               pro   
...                                                 ...          ...   
3649                                                        business   
3651   Hairstylist Salonchezpierre John Casablanca S...     business   
3652       Estilista Profesional Mam Feliz Mis Bebes        business   
3662   Curated Spiritual Wisdom Greater Feeling Love        business   
3663         Witchyaesthetic D O G M O M Cosmetology        business   

       user.zip user.geocity user.geocountry  \
0                    regina,          Canada   
2                                          0   
6                                          0   
7     t1j 0h7    lethbridge,               0   
10                                         0   
...         ...          ...             ...   
3649                                       0   
3651  v1m 2y5       langley,               0   
3652                                       0   
3662                                       0   
3663                                       0   

                               user.email  \
0                                           
2                sally.hawkins@henkel.com   
6                                           
7                                           
10                                          
...                                   ...   
3649              lerasonaanna@icloud.com   
3651             abbyingram98@hotmail.com   
3652  contrerasrojacarlavaleska@gmail.com   
3662                                        
3663                                        

                                      user.website user.phone user.zip_bio  \
0                                                                            
2          http://www.skpevents.co.uk/virtualeduk8                           
6                                                                            
7                          http://edgebarbers.com/                           
10                                                                           
...                                            ...        ...          ...   
3649                                                                         
3651  https://www.vagaro.com/us02/salonchezpierre/                           
3652                                                                         
3662            http://www.linktr.ee/bliinkstudio/                           
3663                                                                         

     user.postal_bio     user.pro_type   user.buis_type        women cut  \
0                     Hair Stylist                  nan  Women's Haircut   
2                     Hair Stylist                  nan  Women's Haircut   
6                     Hair Stylist                  nan  Women's Haircut   
7                     Hair Stylist                  nan  Wom

In [68]:
#Merge
user_profile['user.service'] = user_profile['women cut'] + ', ' + user_profile['men cut'] + ', ' + user_profile['hair color'] + ', ' + user_profile['hair extension'] + ', ' + user_profile['balayage'] + ', ' + user_profile['highlights']

#Drop
user_profile = user_profile.drop(['women cut', 'men cut', 'hair color', 'hair extension', 'balayage', 'highlights'],axis = 1)

user_profile

user.username             user.full_name  \
0                tammycburrows  tammy w | yqr hairstylist   
2     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf   
6          vaghela_bhavesh7799              surendr nagar   
7               edgebarbersyql               edge barbers   
10                  floyd__leo                      floyd   
...                        ...                        ...   
3649                   v_iulia              юля воробьева   
3651          hairbyabbyingram              abby ingram✂️   
3652   valeska_contreras_carla   valeska  contreras rojas   
3662                   __tiale                              
3663                   joyymia                   𝐌̃𝐢̃𝐚̃ ☾   

                                         user.biography user.account  \
0      Canada Saskatchewan Based Hairstylist 30 Year...          pro   
2                          Sales Manager Schwarzkopf             pro   
6                                                                pro   
7      Evolution Barbering Offers Haircuts Beard Tri...          pro   
10                                                               pro   
...                                                 ...          ...   
3649                                                        business   
3651   Hairstylist Salonchezpierre John Casablanca S...     business   
3652       Estilista Profesional Mam Feliz Mis Bebes        business   
3662   Curated Spiritual Wisdom Greater Feeling Love        business   
3663         Witchyaesthetic D O G M O M Cosmetology        business   

       user.zip user.geocity user.geocountry  \
0                    regina,          Canada   
2                                          0   
6                                          0   
7     t1j 0h7    lethbridge,               0   
10                                         0   
...         ...          ...             ...   
3649                                       0   
3651  v1m 2y5       langley,               0   
3652                                       0   
3662                                       0   
3663                                       0   

                               user.email  \
0                                           
2                sally.hawkins@henkel.com   
6                                           
7                                           
10                                          
...                                   ...   
3649              lerasonaanna@icloud.com   
3651             abbyingram98@hotmail.com   
3652  contrerasrojacarlavaleska@gmail.com   
3662                                        
3663                                        

                                      user.website user.phone user.zip_bio  \
0                                                                            
2          http://www.skpevents.co.uk/virtualeduk8                           
6                                                                            
7                          http://edgebarbers.com/                           
10                                                                           
...                                            ...        ...          ...   
3649                                                                         
3651  https://www.vagaro.com/us02/salonchezpierre/                           
3652                                                                         
3662            http://www.linktr.ee/bliinkstudio/                           
3663                                                                         

     user.postal_bio     user.pro_type   user.buis_type  \
0                     Hair Stylist                  nan   
2                     Hair Stylist                  nan   
6                     Hair Stylist                  nan   
7                     Hair Stylist                  nan   
10                    Hair Stylist                  nan   
...       

In [69]:
user_profile['user.service'].unique()

array(["Women's Haircut, , , , , ",
       "Women's Haircut, Men's Haircut, , , , ",
       "Women's Haircut, , , , Balayage, ",
       "Women's Haircut, , , Hair Extensions, , ",
       "Women's Haircut, Men's Haircut, , Hair Extensions, , "],
      dtype=object)

Let's add up all of our tables, and make a csv file to submit!

In [ ]:
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ' '.join(ulist)

a="calvin klein design dress calvin klein"
a=unique_list(a.split())
a

In [70]:
user_profile = user_profile.drop(['user.biography'], axis = 1)
user_profile.head()

user.username             user.full_name user.account  \
0              tammycburrows  tammy w | yqr hairstylist          pro   
2   sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf          pro   
6        vaghela_bhavesh7799              surendr nagar          pro   
7             edgebarbersyql               edge barbers          pro   
10                floyd__leo                      floyd          pro   

     user.zip user.geocity user.geocountry                user.email  \
0                  regina,          Canada                             
2                                        0  sally.hawkins@henkel.com   
6                                        0                             
7   t1j 0h7    lethbridge,               0                             
10                                       0                             

                               user.website user.phone user.zip_bio  \
0                                                                     
2   http://www.skpevents.co.uk/virtualeduk8                           
6                                                                     
7                   http://edgebarbers.com/                           
10                                                                    

   user.postal_bio     user.pro_type user.buis_type               user.service  
0                   Hair Stylist                nan  Women's Haircut, , , , ,   
2                   Hair Stylist                nan  Women's Haircut, , , , ,   
6                   Hair Stylist                nan  Women's Haircut, , , , ,   
7                   Hair Stylist                nan  Women's Haircut, , , , ,   
10                  Hair Stylist                nan  Women's Haircut, , , , ,

In [71]:
user_profile = user_profile.fillna('')
user_profile.head()

user.username             user.full_name user.account  \
0              tammycburrows  tammy w | yqr hairstylist          pro   
2   sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf          pro   
6        vaghela_bhavesh7799              surendr nagar          pro   
7             edgebarbersyql               edge barbers          pro   
10                floyd__leo                      floyd          pro   

     user.zip user.geocity user.geocountry                user.email  \
0                  regina,          Canada                             
2                                        0  sally.hawkins@henkel.com   
6                                        0                             
7   t1j 0h7    lethbridge,               0                             
10                                       0                             

                               user.website user.phone user.zip_bio  \
0                                                                     
2   http://www.skpevents.co.uk/virtualeduk8                           
6                                                                     
7                   http://edgebarbers.com/                           
10                                                                    

   user.postal_bio     user.pro_type user.buis_type               user.service  
0                   Hair Stylist                nan  Women's Haircut, , , , ,   
2                   Hair Stylist                nan  Women's Haircut, , , , ,   
6                   Hair Stylist                nan  Women's Haircut, , , , ,   
7                   Hair Stylist                nan  Women's Haircut, , , , ,   
10                  Hair Stylist                nan  Women's Haircut, , , , ,

In [72]:
user_post['user.username'] = user_post.index
user_post = user_post.reset_index()
user_post = user_post.drop(['input.username','items.caption.text'], axis = 1)
user_post

user.service   user.username
0     Women's Haircut, Men's Haircut, , Hair Extensi...   hairbypetrina
1     Women's Haircut, Men's Haircut, , Hair Extensi...  hairbypeytonxx
2     Women's Haircut, Men's Haircut, , Hair Extensi...    hairbypinkyd
3     Women's Haircut, Men's Haircut, Hair Color, Ha...     hairbypizer
4        Women's Haircut, Men's Haircut, , , Balayage,   hairbypmanning
...                                                 ...             ...
1164             Women's Haircut, Men's Haircut, , , ,        store6546
1165                          Women's Haircut, , , , ,     store_love83
1166  Women's Haircut, Men's Haircut, Hair Color, Ha...       stormiest
1167                          Women's Haircut, , , , ,        story5212
1168     Women's Haircut, Men's Haircut, , , Balayage,         stpaul61

[1169 rows x 2 columns]

In [91]:
user_post = user_post.fillna('')
user_profile = user_profile.fillna('')
user_post = user_post.applymap(str)
user_profile = user_profile.applymap(str)
final = pd.merge(user_profile,user_post, on = ['user.username','user.service'], how = 'outer')
final

user.username             user.full_name user.account  \
0                tammycburrows  tammy w | yqr hairstylist          pro   
1     sallyhawkins_schwarzkopf  sally hawkins-schwarzkopf          pro   
2          vaghela_bhavesh7799              surendr nagar          pro   
3          vaghela_bhavesh7799              surendr nagar          pro   
4          vaghela_bhavesh7799              surendr nagar          pro   
...                        ...                        ...          ...   
4820                 store6546                        NaN          NaN   
4821              store_love83                        NaN          NaN   
4822                 stormiest                        NaN          NaN   
4823                 story5212                        NaN          NaN   
4824                  stpaul61                        NaN          NaN   

     user.zip user.geocity user.geocountry                user.email  \
0                  regina,          Canada                             
1                                        0  sally.hawkins@henkel.com   
2                                        0                             
3                                        0                             
4                                        0                             
...       ...          ...             ...                       ...   
4820      NaN          NaN             NaN                       NaN   
4821      NaN          NaN             NaN                       NaN   
4822      NaN          NaN             NaN                       NaN   
4823      NaN          NaN             NaN                       NaN   
4824      NaN          NaN             NaN                       NaN   

                                 user.website user.phone user.zip_bio  \
0                                                                       
1     http://www.skpevents.co.uk/virtualeduk8                           
2                                                                       
3                                                                       
4                                                                       
...                                       ...        ...          ...   
4820                                      NaN        NaN          NaN   
4821                                      NaN        NaN          NaN   
4822                                      NaN        NaN          NaN   
4823                                      NaN        NaN          NaN   
4824                                      NaN        NaN          NaN   

     user.postal_bio     user.pro_type user.buis_type  \
0                     Hair Stylist                nan   
1                     Hair Stylist                nan   
2                     Hair Stylist                nan   
3                     Hair Stylist                nan   
4                     Hair Stylist                nan   
...              ...               ...            ...   
4820             NaN               NaN            NaN   
4821             NaN               NaN            NaN   
4822             NaN               NaN            NaN   
4823             NaN               NaN            NaN   
4824             NaN               NaN            NaN   

                                           user.service  
0                             Women's Haircut, , , , ,   
1                             Women's Haircut, , , , ,   
2                             Women's Haircut, , , , ,   
3                             Women's Haircut, , , , ,   
4                             Women's Haircut, , , , ,   
...                                                 ...  
4820             Women's Haircut, Men's Haircut, , , ,   
4821                          Women's Haircut, , , , ,   
4822  Women's Haircut, Men's Haircut, Hair Color, Ha...  
4823                          Women's Haircut, , , , ,   
4824     Women's Haircut, Men's Haircut, , , Balayage,   

[482

A little bit more cleaning

In [95]:
#Reordering
cols = ['user.username','user.account', 'user.full_name','user.buis_type', 'user.pro_type', 'user.geocity', 
        'user.geocountry', 'user.zip_bio', 'user.postal_bio', 'user.email', 'user.phone', 'user.website', 'user.service']
final = final[cols]
final

user.username user.account             user.full_name  \
0                tammycburrows          pro  tammy w | yqr hairstylist   
1     sallyhawkins_schwarzkopf          pro  sally hawkins-schwarzkopf   
2          vaghela_bhavesh7799          pro              surendr nagar   
3          vaghela_bhavesh7799          pro              surendr nagar   
4          vaghela_bhavesh7799          pro              surendr nagar   
...                        ...          ...                        ...   
4820                 store6546          NaN                        NaN   
4821              store_love83          NaN                        NaN   
4822                 stormiest          NaN                        NaN   
4823                 story5212          NaN                        NaN   
4824                  stpaul61          NaN                        NaN   

     user.buis_type     user.pro_type user.geocity user.geocountry  \
0               nan  Hair Stylist          regina,          Canada   
1               nan  Hair Stylist                                0   
2               nan  Hair Stylist                                0   
3               nan  Hair Stylist                                0   
4               nan  Hair Stylist                                0   
...             ...               ...          ...             ...   
4820            NaN               NaN          NaN             NaN   
4821            NaN               NaN          NaN             NaN   
4822            NaN               NaN          NaN             NaN   
4823            NaN               NaN          NaN             NaN   
4824            NaN               NaN          NaN             NaN   

     user.zip_bio user.postal_bio                user.email user.phone  \
0                                                                        
1                                  sally.hawkins@henkel.com              
2                                                                        
3                                                                        
4                                                                        
...           ...             ...                       ...        ...   
4820          NaN             NaN                       NaN        NaN   
4821          NaN             NaN                       NaN        NaN   
4822          NaN             NaN                       NaN        NaN   
4823          NaN             NaN                       NaN        NaN   
4824          NaN             NaN                       NaN        NaN   

                                 user.website  \
0                                               
1     http://www.skpevents.co.uk/virtualeduk8   
2                                               
3                                               
4                                               
...                                       ...   
4820                                      NaN   
4821                                      NaN   
4822                                      NaN   
4823                                      NaN   
4824                                      NaN   

                                           user.service  
0                             Women's Haircut, , , , ,   
1                             Women's Haircut, , , , ,   
2                             Women's Haircut, , , , ,   
3                             Women's Haircut, , , , ,   
4                             Women's Haircut, , , , ,   
...                                                 ...  
4820             Women's Haircut, Men's Haircut, , , ,   
4821                          Women's Haircut, , , , ,   
4822  Women's Haircut, Men's Haircut, Hair Color, Ha...  
4823                          Women's Haircut, , , , ,   
4824     Women's Haircut, Men's Haircut, , , Balayage,   

[4825 rows x 13 columns]

In [99]:
#Fill empty with Not found
final = final.fillna('')
final = final.replace('','Not Found')
final = final.replace(' ','Not Found')
final = final.replace('0','Not Found')
final = final.replace('nan','Not Found')

final

user.username user.account             user.full_name  \
0                tammycburrows          pro  tammy w | yqr hairstylist   
1     sallyhawkins_schwarzkopf          pro  sally hawkins-schwarzkopf   
2          vaghela_bhavesh7799          pro              surendr nagar   
3          vaghela_bhavesh7799          pro              surendr nagar   
4          vaghela_bhavesh7799          pro              surendr nagar   
...                        ...          ...                        ...   
4820                 store6546    Not Found                  Not Found   
4821              store_love83    Not Found                  Not Found   
4822                 stormiest    Not Found                  Not Found   
4823                 story5212    Not Found                  Not Found   
4824                  stpaul61    Not Found                  Not Found   

     user.buis_type     user.pro_type user.geocity user.geocountry  \
0         Not Found  Hair Stylist          regina,          Canada   
1         Not Found  Hair Stylist        Not Found       Not Found   
2         Not Found  Hair Stylist        Not Found       Not Found   
3         Not Found  Hair Stylist        Not Found       Not Found   
4         Not Found  Hair Stylist        Not Found       Not Found   
...             ...               ...          ...             ...   
4820      Not Found         Not Found    Not Found       Not Found   
4821      Not Found         Not Found    Not Found       Not Found   
4822      Not Found         Not Found    Not Found       Not Found   
4823      Not Found         Not Found    Not Found       Not Found   
4824      Not Found         Not Found    Not Found       Not Found   

     user.zip_bio user.postal_bio                user.email user.phone  \
0       Not Found       Not Found                 Not Found  Not Found   
1       Not Found       Not Found  sally.hawkins@henkel.com  Not Found   
2       Not Found       Not Found                 Not Found  Not Found   
3       Not Found       Not Found                 Not Found  Not Found   
4       Not Found       Not Found                 Not Found  Not Found   
...           ...             ...                       ...        ...   
4820    Not Found       Not Found                 Not Found  Not Found   
4821    Not Found       Not Found                 Not Found  Not Found   
4822    Not Found       Not Found                 Not Found  Not Found   
4823    Not Found       Not Found                 Not Found  Not Found   
4824    Not Found       Not Found                 Not Found  Not Found   

                                 user.website  \
0                                   Not Found   
1     http://www.skpevents.co.uk/virtualeduk8   
2                                   Not Found   
3                                   Not Found   
4                                   Not Found   
...                                       ...   
4820                                Not Found   
4821                                Not Found   
4822                                Not Found   
4823                                Not Found   
4824                                Not Found   

                                           user.service  
0                             Women's Haircut, , , , ,   
1                             Women's Haircut, , , , ,   
2                             Women's Haircut, , , , ,   
3                             Women's Haircut, , , , ,   
4                             Women's Haircut, , , , ,   
...                                                 ...  
4820             Women's Haircut, Men's Haircut, , , ,   
4821                          Women's Haircut, , , , ,   
4822  Women's Haircut, Men's Haircut, Hair Color, Ha...  
4823                          Women's Haircut, , , , ,   
4824     Women's Haircut, Men's Haircut, , , Balayage,   

[4825 rows x 13 columns]

Finally, let's export it to a csv for submission.

In [97]:
final.to_csv('Submission.csv')

## End
Thank you for taking a look at my submission as part of my job application to SalonEverywhere. I learned a lot about Data Cleaning for NLP, as well as how to use certain libraries such as Regular Expression, NLTK, and geograpy.